In [144]:
import os
import imageio
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage, misc

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier



In [2]:
letterDict={
  "a": 0,"b": 1,"c": 2,"d": 3,"e": 4,"f": 5,"g": 6,"h": 7,"i": 8,
    "j": 9,"k": 10,"l": 11,"m": 12,"n": 13,"o": 14,"p": 15,"q": 16,"r": 17,
    "s": 18,"t": 19,"u": 20,"v": 21,"w": 22,"x": 23,"y": 24,"z": 25}

numberDict={
    0:"A", 1:"B", 2:"C", 3:"D", 4:"E", 5:"F", 6:"G", 7:"H", 8:"I",
    9:"J", 10:"K", 11:"L", 12:"M", 13:"N", 14:"O", 15:"P", 16:"Q", 17:"R",
    18:"S", 19:"T", 20:"U", 21:"V", 22:"W", 23:"X", 24:"Y", 25:"Z"}


def readPicture(df_complete, directory,letter_value):
    im = imageio.imread(directory)
    im.shape
    print("Letter value: ", directory, end="\r")

    im=np.reshape(im,400)
    df = pd.DataFrame(data=im).T
    y = np.repeat(0,400)
    df['Letter'] = letter_value
    
    return df_complete.append(df, ignore_index=True)

#Method ment for adding augmented pictures to a dataset
def addPictureToDataset(dataset, picture, letter_value):
    picture = np.reshape(picture,400)
    df = pd.DataFrame(data=picture).T
    df['Letter'] = letter_value
    return dataset.append(df, ignore_index=True)



In [3]:
## *** Data augmentation *** ##

# Noise:



In [3]:
def read_all_images(upper_directory):
    number_of_images = 0
    original_imgs = pd.DataFrame()
    aug20 = pd.DataFrame()
    aug90 = pd.DataFrame()
    aug270 = pd.DataFrame()
    aug320 = pd.DataFrame()
    for subdir, dirs, files in os.walk(upper_directory):
        for file in files:
            number_of_images+=1
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if filepath.endswith(".jpg"):
                # print(filepath)
                # Rotate every image that is being loaded:
            
                x = filepath.split("/")
                rotate20 = rotate_image(filepath,x[-1],20)
                rotate90 = rotate_image(filepath,x[-1],90)
                rotate270 = rotate_image(filepath,x[-1],270)
                rotate320 = rotate_image(filepath,x[-1],320)
                                
                # The label for the data:
                letter = (x[-1][0])
                letter_val = letterDict.get(letter)
                original_imgs = readPicture(original_imgs,filepath,letter_val)
                aug20 = addPictureToDataset(aug20, rotate20, letter_val)
                aug90 = addPictureToDataset(aug90, rotate90, letter_val)
                aug270 = addPictureToDataset(aug270, rotate270, letter_val)
                aug320 = addPictureToDataset(aug320, rotate320, letter_val)
    print("image")
    # vv.imshow("./dataset3/chars74k-lite/augmented/rotated_neg_a102.jpg")
    print("number_of_images", number_of_images)
    return original_imgs, aug20, aug90, aug270, aug320


#Rotates an image, stores it in a folder and returns the rotated image
def rotate_image(image_path,ending,deg):
    out_path = "./dataset3/augmented/"
    image_to_rotate = plt.imread(image_path)
    
    # Rotating the image:
    rotated_positive = ndimage.rotate(image_to_rotate, deg, reshape=False)
   
    fullpath = os.path.join(out_path, 'rotated_'+ str(deg) + '_' + ending)
    imageio.imwrite(fullpath, rotated_positive)
    return rotated_positive
    
    
               
# Unaugmented data, no feature selection:            
original_raw, aug20, aug90, aug270, aug320 = read_all_images("./dataset3/chars74k-lite")
original_raw
               


imager value:  ./dataset3/chars74k-lite/z/z_9.jpggg./dataset3/chars74k-lite/a/a_434.jpg ./dataset3/chars74k-lite/a/a_630.jpg ./dataset3/chars74k-lite/b/b_56.jpg ./dataset3/chars74k-lite/d/d_9.jpg ./dataset3/chars74k-lite/e/e_106.jpg ./dataset3/chars74k-lite/e/e_33.jpg ./dataset3/chars74k-lite/e/e_350.jpg ./dataset3/chars74k-lite/g/g_107.jpg ./dataset3/chars74k-lite/g/g_140.jpg ./dataset3/chars74k-lite/h/h_119.jpg ./dataset3/chars74k-lite/h/h_191.jpg ./dataset3/chars74k-lite/i/i_243.jpg ./dataset3/chars74k-lite/i/i_344.jpg ./dataset3/chars74k-lite/i/i_69.jpg ./dataset3/chars74k-lite/j/j_4.jpg ./dataset3/chars74k-lite/n/n_25.jpg ./dataset3/chars74k-lite/n/n_324.jpg ./dataset3/chars74k-lite/n/n_341.jpg ./dataset3/chars74k-lite/o/o_283.jpg ./dataset3/chars74k-lite/o/o_507.jpg ./dataset3/chars74k-lite/r/r_211.jpg ./dataset3/chars74k-lite/r/r_233.jpg ./dataset3/chars74k-lite/r/r_271.jpg ./dataset3/chars74k-lite/r/r_97.jpg ./dataset3/chars74k-lite/s/s_240.jpg ./dataset3/chars74k-lite/t/t_392.

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,Letter
0,193,175,177,191,177,188,194,177,197,157,...,175,191,175,183,188,225,185,167,193,0
1,37,34,37,46,50,45,40,39,39,38,...,34,34,35,36,37,36,36,37,38,0
2,25,33,40,42,40,38,36,35,42,46,...,33,38,38,32,31,51,43,41,41,0
3,27,34,36,30,31,35,32,23,20,12,...,24,23,23,27,40,8,27,33,20,0
4,186,192,175,189,202,182,181,188,191,193,...,194,196,185,182,196,193,190,191,194,0
5,34,37,7,25,14,19,22,26,34,13,...,14,15,19,20,17,8,30,21,15,0
6,40,5,5,0,34,8,16,38,22,40,...,35,12,0,57,31,29,6,47,59,0
7,232,251,223,238,237,221,235,232,231,216,...,20,37,35,42,22,56,37,41,13,0
8,232,228,228,229,230,234,212,138,45,45,...,232,235,199,69,26,34,54,49,96,0
9,223,229,220,216,226,222,217,231,238,225,...,229,234,228,222,223,227,227,227,227,0


In [4]:
#Methood merges dataset1 with dataset2
def mergeData(dataset1, dataset2):
    return dataset1.append(dataset2, ignore_index=True)

mergedData = mergeData(original_raw, aug20)
mergedData = mergeData(mergedData, aug90)
mergedData = mergeData(mergedData, aug270)
mergedData = mergeData(mergedData, aug320)
mergedData

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,Letter
0,193,175,177,191,177,188,194,177,197,157,...,175,191,175,183,188,225,185,167,193,0
1,37,34,37,46,50,45,40,39,39,38,...,34,34,35,36,37,36,36,37,38,0
2,25,33,40,42,40,38,36,35,42,46,...,33,38,38,32,31,51,43,41,41,0
3,27,34,36,30,31,35,32,23,20,12,...,24,23,23,27,40,8,27,33,20,0
4,186,192,175,189,202,182,181,188,191,193,...,194,196,185,182,196,193,190,191,194,0
5,34,37,7,25,14,19,22,26,34,13,...,14,15,19,20,17,8,30,21,15,0
6,40,5,5,0,34,8,16,38,22,40,...,35,12,0,57,31,29,6,47,59,0
7,232,251,223,238,237,221,235,232,231,216,...,20,37,35,42,22,56,37,41,13,0
8,232,228,228,229,230,234,212,138,45,45,...,232,235,199,69,26,34,54,49,96,0
9,223,229,220,216,226,222,217,231,238,225,...,229,234,228,222,223,227,227,227,227,0


In [6]:
# splitting function:
def get_train_test_set(data):
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()
    
    for i in range(26):
        #print(data.loc[data['Letter']==i])
        train, test = train_test_split(data.loc[data['Letter']==i], 
                                       test_size=0.2, shuffle=True)
        
        #print(train)
        train_set = train_set.append([train], ignore_index=True)
        test_set = test_set.append([test], ignore_index=True)
    return train_set, test_set

## Running models on the data:
#### KNN:

In [163]:
#### Running models on the raw data:



def calc_predictions(classifier, train):
    y_pred = []
    n = train.shape[0]
    counter = 0
    clf = classifier.fit(train, train.iloc[:,-1])
    for row in train.iterrows():
        y_pred.append(clf.predict([row[1].tolist()]))
        counter +=1
        print((n/counter)*100,"%")
    return y_pred,clf

def test_classifier(clf, data):
    y = []
    for row in data.iterrows():
        y.append(clf.predict(([row[1].tolist()])))
    return y



In [ ]:

# Raw training and test set:
train_set_raw, test_set_raw = get_train_test_set(original_raw)

#### KNN #####
neigh = KNeighborsClassifier(n_neighbors=3)
y_raw_train_pred, clf = calc_predictions(neigh,train_set_raw)

y_test_pred = test_classifier(clf, test_set_raw)

In [150]:
### Running model on the augmented + raw data:

train_set_merged, test_set_merged = get_train_test_set(mergedData)

neigh = KNeighborsClassifier(n_neighbors=3)
y_merged_train_pred, clf = calc_predictions(neigh, train_set_merged)

y_merged_test_pred = test_classifier(clf, test_set_merged)


### Accuracy scores for knn
#### Training data:

In [153]:
## Raw data:
# Train:
print("accuracy_score_knn_training_data:", accuracy_score( train_set_raw.iloc[:,-1],y_raw_train_pred))
# Test:
print("accuracy_score_knn_test_data:", accuracy_score( test_set_raw.iloc[:,-1],y_test_pred))


print(train_set_merged.iloc[:,-1].shape)

## Merged data:
# Train:
print("accuracy_score_knn_training_data:", accuracy_score( train_set_merged.iloc[:,-1],y_merged_train_pred))
# Test
print("accuracy_score_knn_test_data:", accuracy_score( test_set_merged.iloc[:,-1],y_merged_test_pred))



accuracy_score_knn_training_data: 0.7834507042253521
accuracy_score_knn_test_data: 0.6801675977653632
(28448,)
accuracy_score_knn_training_data: 0.7734462879640045
accuracy_score_knn_test_data: 0.6848987626546682


## Random Forest- Not implemented yet!


In [164]:
clf_rf = RandomForestClassifier(n_estimators=1000,random_state=0)

y_raw_train_pred_rf, clf_rf = calc_predictions(clf_rf,train_set_raw)

y_test_pred_rf = test_classifier(clf_rf, test_set_raw)

568000.0 %
284000.0 %
189333.3333333333 %
142000.0 %
113600.0 %
94666.66666666666 %
81142.85714285714 %
71000.0 %
63111.11111111111 %
56800.0 %
51636.36363636364 %
47333.33333333333 %
43692.30769230769 %
40571.42857142857 %
37866.66666666667 %
35500.0 %
33411.76470588236 %
31555.555555555555 %
29894.736842105267 %
28400.0 %
27047.61904761905 %
25818.18181818182 %
24695.652173913044 %
23666.666666666664 %
22720.0 %
21846.153846153844 %
21037.037037037036 %
20285.714285714286 %
19586.206896551725 %
18933.333333333336 %
18322.580645161288 %
17750.0 %
17212.121212121212 %
16705.88235294118 %
16228.571428571428 %
15777.777777777777 %
15351.351351351352 %
14947.368421052633 %
14564.102564102563 %
14200.0 %
13853.658536585366 %
13523.809523809525 %
13209.302325581397 %
12909.09090909091 %
12622.222222222223 %
12347.826086956522 %
12085.106382978724 %
11833.333333333332 %
11591.836734693878 %
11360.0 %
11137.254901960785 %
10923.076923076922 %
10716.981132075472 %
10518.518518518518 %
10327.27

1368.6746987951806 %
1365.3846153846152 %
1362.1103117505995 %
1358.8516746411483 %
1355.6085918854417 %
1352.3809523809523 %
1349.1686460807603 %
1345.9715639810427 %
1342.789598108747 %
1339.622641509434 %
1336.4705882352941 %
1333.3333333333335 %
1330.2107728337237 %
1327.1028037383178 %
1324.009324009324 %
1320.9302325581396 %
1317.8654292343388 %
1314.8148148148148 %
1311.7782909930716 %
1308.7557603686635 %
1305.7471264367816 %
1302.7522935779816 %
1299.771167048055 %
1296.8036529680364 %
1293.8496583143508 %
1290.9090909090908 %
1287.9818594104308 %
1285.0678733031675 %
1282.1670428893904 %
1279.2792792792793 %
1276.4044943820224 %
1273.542600896861 %
1270.6935123042506 %
1267.857142857143 %
1265.033407572383 %
1262.2222222222222 %
1259.4235033259424 %
1256.637168141593 %
1253.8631346578366 %
1251.101321585903 %
1248.3516483516485 %
1245.6140350877195 %
1242.8884026258206 %
1240.174672489083 %
1237.4727668845314 %
1234.7826086956522 %
1232.1041214750542 %
1229.4372294372295 %
12

686.819830713422 %
685.9903381642513 %
685.1628468033776 %
684.3373493975903 %
683.5138387484958 %
682.6923076923076 %
681.8727490996398 %
681.0551558752998 %
680.2395209580839 %
679.4258373205741 %
678.6140979689367 %
677.8042959427208 %
676.9964243146603 %
676.1904761904761 %
675.3864447086802 %
674.5843230403801 %
673.7841043890866 %
672.9857819905213 %
672.189349112426 %
671.3947990543735 %
670.6021251475797 %
669.811320754717 %
669.0223792697291 %
668.2352941176471 %
667.4500587544065 %
666.6666666666667 %
665.8851113716296 %
665.1053864168618 %
664.327485380117 %
663.5514018691589 %
662.777129521587 %
662.004662004662 %
661.2339930151338 %
660.4651162790698 %
659.698025551684 %
658.9327146171694 %
658.1691772885284 %
657.4074074074074 %
656.6473988439307 %
655.8891454965358 %
655.1326412918108 %
654.3778801843317 %
653.6248561565017 %
652.8735632183908 %
652.1239954075776 %
651.3761467889908 %
650.6300114547537 %
649.8855835240275 %
649.1428571428571 %
648.4018264840182 %
647.662

457.6954069298952 %
457.3268921095008 %
456.9589702333065 %
456.5916398713826 %
456.22489959839356 %
455.8587479935794 %
455.4931836407378 %
455.1282051282051 %
454.7638110488391 %
454.4 %
454.03677058353315 %
453.6741214057508 %
453.31205107741425 %
452.9505582137161 %
452.589641434263 %
452.22929936305735 %
451.86953062848045 %
451.51033386327504 %
451.1517077045274 %
450.7936507936508 %
450.43616177636795 %
450.07923930269413 %
449.72288202692 %
449.36708860759495 %
449.0118577075099 %
448.6571879936809 %
448.30307813733225 %
447.9495268138801 %
447.5965327029157 %
447.244094488189 %
446.89221085759243 %
446.5408805031447 %
446.1901021209741 %
445.83987441130296 %
445.4901960784314 %
445.14106583072095 %
444.7924823805795 %
444.44444444444446 %
444.0969507427678 %
443.75 %
443.4035909445745 %
443.0577223088924 %
442.7123928293063 %
442.36760124610595 %
442.02334630350197 %
441.6796267496112 %
441.33644133644134 %
440.99378881987576 %
440.6516679596587 %
440.31007751937983 %
439.9690

344.66019417475724 %
344.45118253486964 %
344.24242424242425 %
344.0339188370685 %
343.82566585956414 %
343.61766485178464 %
343.409915356711 %
343.202416918429 %
342.99516908212564 %
342.7881713940857 %
342.5814234016888 %
342.3749246534057 %
342.16867469879514 %
341.9626730885009 %
341.7569193742479 %
341.5514131088394 %
341.3461538461538 %
341.14114114114113 %
340.9363745498199 %
340.73185362927416 %
340.5275779376499 %
340.32354703415217 %
340.11976047904193 %
339.9162178336326 %
339.7129186602871 %
339.50986252241483 %
339.30704898446834 %
339.1044776119403 %
338.9021479713604 %
338.7000596302922 %
338.49821215733016 %
338.2966051220965 %
338.0952380952381 %
337.8941106484235 %
337.6932223543401 %
337.4925727866904 %
337.29216152019006 %
337.0919881305638 %
336.8920521945433 %
336.69235328986366 %
336.49289099526067 %
336.29366489046777 %
336.094674556213 %
335.8959195742164 %
335.69739952718675 %
335.49911399881864 %
335.30106257378986 %
335.10324483775815 %
334.9056603773585 %
3

...

140.21229326092325 %
140.177690029615 %
140.14310387367382 %
140.10853478046374 %
140.0739827373613 %
140.03944773175542 %
140.00492975104757 %
139.97042878265157 %
139.9359448139936 %
139.9014778325123 %
139.8670278256587 %
139.8325947808961 %
139.7981786857002 %
139.76377952755905 %
139.72939729397294 %
139.6950319724545 %
139.66068355052866 %
139.62635201573255 %
139.59203735561562 %
139.55773955773955 %
139.52345860967822 %
139.48919449901769 %
139.45494721335626 %
139.42071674030436 %
139.38650306748465 %
139.3523061825319 %
139.31812607309294 %
139.28396272682687 %
139.24981613140474 %
139.2156862745098 %
139.1815731438373 %
139.14747672709456 %
139.11339701200097 %
139.07933398628796 %
139.0452876376989 %
139.01125795398923 %
138.97724492292636 %
138.94324853228963 %
138.90926876987038 %
138.8753056234719 %
138.8413590809093 %
138.80742913000978 %
138.77351575861226 %
138.73961895456765 %
138.7057387057387 %
138.671875 %
138.63802782523797 %
138.6041971693509 %
138.5703830202488

127.6978417266187 %
127.6691391323893 %
127.64044943820225 %
127.61177263536283 %
127.5831087151842 %
127.55445766898721 %
127.52581948810058 %
127.49719416386083 %
127.46858168761221 %
127.43998205070675 %
127.41139524450426 %
127.38282126037228 %
127.3542600896861 %
127.32571172382873 %
127.29717615419094 %
127.26865337217119 %
127.24014336917561 %
127.21164613661816 %
127.1831616659203 %
127.15468994851132 %
127.12623097582811 %
127.09778473931527 %
127.06935123042507 %
127.0409304406173 %
127.01252236135959 %
126.98412698412697 %
126.95574430040233 %
126.92737430167598 %
126.89901697944595 %
126.87067232521778 %
126.8423403305047 %
126.81402098682742 %
126.78571428571428 %
126.75742021870118 %
126.72913877733154 %
126.70086995315637 %
126.67261373773417 %
126.64437012263099 %
126.61613909942042 %
126.58792065968353 %
126.55971479500892 %
126.53152149699265 %
126.5033407572383 %
126.47517256735694 %
126.44701691896705 %
126.41887380369464 %
126.39074321317312 %
126.36262513904337 %


117.30689797604296 %
117.28267602725583 %
117.25846407927332 %
117.234262125903 %
117.21007016095749 %
117.1858881782546 %
117.16171617161717 %
117.13755413487317 %
117.11340206185568 %
117.0892599464028 %
117.06512778235778 %
117.04100556356893 %
117.01689328388957 %
116.99279093717816 %
116.96869851729818 %
116.94461601811818 %
116.92054343351172 %
116.89648075735748 %
116.8724279835391 %
116.84838510594528 %
116.82435211846978 %
116.80032901501131 %
116.7763157894737 %
116.75231243576567 %
116.72831894780107 %
116.70433531949867 %
116.68036154478226 %
116.65639761758062 %
116.6324435318275 %
116.6084992814617 %
116.58456486042692 %
116.56064026267185 %
116.53672548215019 %
116.5128205128205 %
116.48892534864643 %
116.46503998359647 %
116.44116441164411 %
116.41729862676777 %
116.39344262295081 %
116.36959639418151 %
116.34575993445308 %
116.32193323776367 %
116.2981162981163 %
116.27430910951892 %
116.25051166598443 %
116.2267239615306 %
116.20294599018004 %
116.17917774596032 %
116

108.45904143593661 %
108.43833524245896 %
108.4176369536171 %
108.3969465648855 %
108.37626407174203 %
108.35558946966806 %
108.33492275414838 %
108.31426392067125 %
108.2936129647283 %
108.27296988181472 %
108.25233466742901 %
108.23170731707317 %
108.21108782625262 %
108.19047619047619 %
108.16987240525616 %
108.14927646610815 %
108.1286883685513 %
108.10810810810811 %
108.08753568030447 %
108.06697108066972 %
108.04641430473654 %
108.02586534804108 %
108.00532420612284 %
107.9847908745247 %
107.96426534879299 %
107.94374762447738 %
107.92323769713093 %
107.90273556231003 %
107.88224121557455 %
107.86175465248766 %
107.84127586861591 %
107.82080485952923 %
107.80034162080092 %
107.77988614800759 %
107.75943843672928 %
107.7389984825493 %
107.71856628105442 %
107.69814182783466 %
107.67772511848341 %
107.65731614859742 %
107.63691491377676 %
107.61652140962485 %
107.59613563174844 %
107.57575757575756 %
107.55538723726568 %
107.53502461188944 %
107.51466969524893 %
107.49432248296745 

100.87018291600072 %
100.85227272727273 %
100.8343688975679 %
100.81647142350019 %
100.79858030168589 %
100.78069552874378 %
100.76281710129503 %
100.74494501596311 %
100.727079269374 %
100.70921985815602 %
100.69136677893991 %
100.67352002835874 %
100.65567960304801 %
100.63784549964565 %
100.62001771479186 %
100.60219624512929 %
100.58438108730299 %
100.56657223796034 %
100.54876969375111 %
100.53097345132744 %
100.51318350734384 %
100.49539985845719 %
100.47762250132672 %
100.45985143261409 %
100.44208664898319 %
100.42432814710043 %
100.40657592363445 %
100.38882997525627 %
100.37109029863933 %
100.35335689045937 %
100.33562974739445 %
100.31790886612504 %
100.30019424333392 %
100.2824858757062 %
100.2647837599294 %
100.24708789269326 %
100.22939827068996 %
100.21171489061398 %
100.1940377491621 %
100.1763668430335 %
100.15870216892964 %
100.1410437235543 %
100.1233915036136 %
100.10574550581602 %
100.08810572687226 %
100.07047216349542 %
100.0528448124009 %
100.03522367030644 %
10

In [ ]:
y_merged_train_pred_rf, clf = calc_predictions(clf_rf, train_set_merged)

y_merged_test_pred_rf = test_classifier(clf, test_set_merged)


In [165]:
### Random forest accuracy:
## Raw data:
# Train:
print("accuracy_score_knn_training_data:", accuracy_score( train_set_raw.iloc[:,-1],y_raw_train_pred_rf))
# Test:
print("accuracy_score_knn_test_data:", accuracy_score( test_set_raw.iloc[:,-1],y_test_pred_rf))


#print(train_set_merged.iloc[:,-1].shape)
## Merged data:

# Train:
 print("accuracy_score_knn_training_data_merged:", accuracy_score( train_set_merged.iloc[:,-1],y_merged_train_pred_rf))
# Test
 print("accuracy_score_knn_test_data_merged:", accuracy_score( test_set_merged.iloc[:,-1],y_merged_test_pred_rf))



accuracy_score_knn_training_data: 1.0
accuracy_score_knn_test_data: 0.7863128491620112


In [ ]:
## kladd:

# cm_knn = confusion_matrix(y_test, y_pred_knn)
# plt.imshow(cm_knn, cmap='spring', interpolation='None')
# plt.show()



#print("Confusion matrix kNN:")
#print(cm_knn)
#print(" ")
#print("accuracy_score_knn:", accuracy_score(y_test,y_pred_knn))


def printLetter(letter):
    numpy_letter = letter[0:-1].values
    plt.imshow(numpy_letter.reshape(20,20), cmap = matplotlib.cm.binary)
    plt.axis("off")
    plt.title('Supposed to be {}'.format(numberDict.get(letter['Letter'])), fontsize=20)
    plt.show()
    return

a_letter = train_set.iloc[2001]
printLetter(a_letter)
